# 0. Library & Data

In [1]:
import pandas as pd
import json
import re
import os
import requests 
import zipfile 
import io 

from konlpy.tag import Okt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report
from typing import Dict, Any, Callable, Tuple, List, Optional

import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_data(url: str = 'data/') -> Tuple[pd.DataFrame, pd.DataFrame]:
    """훈련 및 테스트 데이터 파일을 로드하고 결합합니다. 파일이 없으면 GitHub에서 다운로드합니다."""
    print("1. 데이터 로드 시작...")
    
    # 필수 파일 존재 여부 확인
    # 파일 이름은 수정할 것
    required_files = ['train.csv', 'general_dialog1.csv', 'general_dialog2.csv', 'test.json']
    all_files_exist = all(os.path.exists(os.path.join(url, f)) for f in required_files)

    if not all_files_exist:
        print("필수 데이터 파일이 없습니다. GitHub에서 다운로드합니다...")
        download_url = "https://github.com/tunib-ai/DKTC/archive/refs/heads/main.zip"
        
        try:
            # data 디렉토리가 없으면 생성
            os.makedirs(url, exist_ok=True)
            
            print("데이터 다운로드 중...")
            response = requests.get(download_url)
            response.raise_for_status() # HTTP 오류 발생 시 예외 발생

            print("다운로드 완료, 압축 해제 중...")
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                for file in z.namelist():
                    # DKTC-main/data/ 폴더 안 파일만 추출
                    if file.startswith("DKTC-main/data/") and not file.endswith('/'):
                        # 파일 경로 재설정: 'DKTC-main/data/파일명' -> 'data/파일명'
                        target_path = os.path.join(url, os.path.relpath(file, "DKTC-main/data"))
                        os.makedirs(os.path.dirname(target_path), exist_ok=True)
                        with z.open(file) as source, open(target_path, "wb") as target:
                            target.write(source.read())
            print(f"'{url}' 폴더에 데이터 추출 완료!")
        except requests.exceptions.RequestException as e:
            print(f"데이터 다운로드 실패: HTTP 오류 또는 연결 문제. {e}")
            raise FileNotFoundError("데이터를 다운로드할 수 없습니다. 수동으로 파일을 넣어주세요.")
        except Exception as e:
            print(f"파일 압축 해제/저장 중 오류 발생: {e}")
            raise FileNotFoundError("다운로드 후 파일 저장에 실패했습니다.")


    # --- 데이터 로드 진행 (파일이 존재하거나 다운로드 후) ---
    train = pd.read_csv(os.path.join(url, 'train.csv'))
    a = pd.read_csv(os.path.join(url, 'general_dialog1.csv')).rename(columns={'dialogue': 'conversation'})
    a['class'] = '일반 대화'
    a['idx'] = range(0,len(a))
    a = a[['idx','class','conversation']]
    b = pd.read_csv(os.path.join(url, 'general_dialog2.csv'))
    b = b[b['class'] == "일반 대화"] 
    train_df = pd.concat([train, a, b], axis=0, ignore_index=True).drop(columns='idx', errors='ignore')

    # 테스트 데이터 로드 (test.json)
    with open(os.path.join(url, "test.json"), "r", encoding="utf-8") as f:
        test_df = pd.DataFrame({"conversation": [v["text"] for v in json.load(f).values()]})
    
    print(f"   - 훈련 데이터 shape: {train_df.shape}")
    print(f"   - 테스트 데이터 shape: {test_df.shape}")
    return train_df, test_df

# 1. Preprocessing & Encoding

In [3]:
def preprocess_data(train_df: pd.DataFrame, test_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, Dict[int, str]]:
    """텍스트 데이터를 정제하고 요청된 고정된 숫자 레이블로 인코딩합니다."""
    print("2. 데이터 전처리 시작...")
    
    # 전처리 함수: 특수 문자 및 다중 공백 제거 (생략)
    def clean_text(text: str) -> str:
        text = str(text) 
        text = re.sub(r'[^\w\s]', ' ', text, flags=re.UNICODE) 
        text = re.sub(r'\s+', ' ', text).strip()
        return text 

    train_df['cleaned_conversation'] = train_df['conversation'].apply(clean_text)
    test_df['cleaned_conversation'] = test_df['conversation'].apply(clean_text)

    LABEL_MAPPING = {
        '협박 대화': 0,
        '갈취 대화': 1,
        '직장 내 괴롭힘 대화': 2,
        '기타 괴롭힘 대화': 3,
        '일반 대화': 4
    }
    
    # 인코딩 수행
    train_df['label_encoded'] = train_df['class'].map(LABEL_MAPPING)
    
    print(f"   - 레이블 클래스: {len(LABEL_MAPPING)}개")
    return train_df, test_df

In [4]:
okt = Okt()

def get_okt_tokenizer(mode: str = 'morphs') -> Callable[[str], List[str]]:
    """Okt 객체와 모드를 기반으로 토크나이저 함수를 반환합니다."""
    if mode == 'morphs':
        def okt_tokenizer(text: str) -> list:
            if pd.isna(text): return []
            return okt.morphs(text) # 형태소 전체 사용
    elif mode == 'nouns':
        def okt_tokenizer(text: str) -> list:
            if pd.isna(text): return []
            return okt.nouns(text)  # 명사만 사용
    else:
        raise ValueError(f"지원하지 않는 Okt 모드: {mode}. 'morphs' 또는 'nouns'를 사용하세요.")
        
    return okt_tokenizer

def simple_tokenizer(text: str) -> list:
    """간단한 공백 기반 토크나이저 (예시)"""
    return text.split() if isinstance(text, str) else []

OpenJDK 64-Bit Server VM warning: Attempt to protect stack guard pages failed.
OpenJDK 64-Bit Server VM warning: Attempt to deallocate stack guard pages failed.


In [5]:
def encode_data_tfidf(train_df: pd.DataFrame, test_df: pd.DataFrame, 
                      tokenizer_name: str, okt_mode: Optional[str] = 'morphs') -> Tuple[Any, Any, Any, Any, Any, TfidfVectorizer]:
    """TfidfVectorizer를 사용하여 텍스트 데이터를 벡터로 인코딩합니다."""
    print("3. Tfidf Encoding 시작...")
    
    if tokenizer_name == 'okt':
        tokenizer = get_okt_tokenizer(mode=okt_mode)
        print(f"   - Tfidf 토크나이저 사용: Okt (Mode: {okt_mode})")
    elif tokenizer_name == 'simple':
        tokenizer = simple_tokenizer
        print(f"   - Tfidf 토크나이저 사용: Simple (공백 분리)")
    else:
        raise ValueError(f"지원하지 않는 토크나이저: {tokenizer_name}")

    tfidf_vect = TfidfVectorizer(tokenizer=tokenizer)
    
    X = train_df['cleaned_conversation']
    y = train_df['label_encoded']
    
    X_tfidf = tfidf_vect.fit_transform(X)
    
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_tfidf, y, test_size=0.2, random_state=42, stratify=y
    )
    
    X_test_final = tfidf_vect.transform(test_df['cleaned_conversation'])
    
    print(f"   - **특징(컬럼) 개수 (Vocabulary Size): {X_tfidf.shape[1]}**")
    print(f"   - 학습용 데이터 shape: {X_tr.shape}")
    print(f"   - 검증용 데이터 shape: {X_val.shape}")
    
    return X_tr, X_val, y_tr, y_val, X_test_final, tfidf_vect

# 2. ML Train & F1-score

In [6]:
def train_and_evaluate_model(model_name: str, model_class: Callable, model_params: Dict[str, Any], 
                             X_tr, X_val, y_tr, y_val) -> Dict[str, Any]:
    """특정 모델로 학습을 진행하고 검증 데이터로 성능을 평가합니다. XGBoost CPU/GPU 설정 포함."""
    
    if model_name.startswith('XGBoost'):
        if 'tree_method' not in model_params:
                # 1. tree_method가 명시되지 않은 경우: 'hist'로 강제 설정 (XGBoostError 방지)
                model_params['tree_method'] = 'hist' 
                print("   - XGBoost: tree_method가 명시되지 않아 CPU 기반 'hist' 사용.")
        else:
                # 2. tree_method가 명시된 경우: 사용자가 설정한 값을 출력
                method = model_params['tree_method']
                if method == 'gpu_hist':
                    # 사용자가 명시적으로 'gpu_hist'를 설정하면 오류 가능성을 안내
                    print(f"   - XGBoost: GPU 가속 ('{method}')을 사용합니다. (GPU 미컴파일 시 오류 발생 가능)")
                elif method == 'hist':
                    print(f"   - XGBoost: CPU 기반 병렬 처리 ('{method}')을 사용합니다.")
                
        model_params.setdefault('use_label_encoder', False)
        model_params.setdefault('eval_metric', 'mlogloss')

            
    print(f"\n 모델 학습 및 평가: {model_name} 시작...")

    model = model_class(**model_params)
    model.fit(X_tr, y_tr)
    
    y_pred = model.predict(X_val)
    
    f1_micro = f1_score(y_val, y_pred, average='micro')
    report = classification_report(y_val, y_pred)
    
    print(f"   - F1 Micro Score: {f1_micro:.4f}")
    print("\n[ Classification Report ]")
    print(report)
    
    return {
        'model_name': model_name,
        'model': model,
        'f1_micro': f1_micro,
        'classification_report': report
    }

In [7]:
def run_experiment_flexible(data_path: str = './data/', 
                            tokenizer_name: str = 'okt',
                            okt_mode: Optional[str] = 'morphs',
                            experiments_config: List[Dict[str, Any]] = None):
    """전체 ML 파이프라인을 유연한 설정으로 실행합니다."""
    
    print(" ML 파이프라인 실행 시작 ")
    
    try:
        # 데이터 로드 및 전처리
        train_df, test_df = load_data(url=data_path)
        train_df, test_df = preprocess_data(train_df, test_df)
        
        # Tfidf 인코딩
        X_tr, X_val, y_tr, y_val, X_test_final, tfidf_vect = encode_data_tfidf(
            train_df, test_df, tokenizer_name, okt_mode
        )

    except FileNotFoundError:
        print("\n 데이터 파일을 찾을 수 없습니다. 경로를 확인해 주세요.")
        return
    except Exception as e:
        print(f"\n 데이터 로드/전처리 중 오류 발생: {e}")
        return
        
    # 기본 실험 설정
    if experiments_config is None:
        experiments_config = [
            # BaseLine이라 random_state 제외한 나머지 값은 전부 default로 진행
            # 다만 LR에 경우, 기존에는 이진분류 모델이기에 파라미터로 solver를 liblinear로 변경하여 다중분류로 변경
            {'name': 'LR_Baseline', 'class': LogisticRegression, 'params': {'random_state': 24, 'solver': 'liblinear'}},
            {'name': 'RF_Baseline', 'class': RandomForestClassifier, 'params': {'random_state': 24}},
            {'name': 'XGBoost_Baseline', 'class': XGBClassifier, 'params': {'random_state': 24}} 
        ]

    results = []
    
    # 3. 모델별 학습 및 평가 실행
    for exp in experiments_config:
        result = train_and_evaluate_model(
            exp['name'], exp['class'], exp['params'].copy(), X_tr, X_val, y_tr, y_val
        )
        results.append(result)

    # 4. 최종 결과 요약
    print("\n==============================================")
    print("최종 실험 결과 요약 (F1 Micro)")
    for res in results:
        print(f"- **{res['model_name']}**: **{res['f1_micro']:.4f}**")
        
    return results, X_test_final

# 3. PipeLine & Submission

In [8]:
if __name__ == "__main__":
    
    # 🧪 실험 1: Okt (morphs) 사용 - 기본 설정
    print("--- 1차 실험: Okt (morphs) 사용 ---")
    results_morphs, X_test_final_morphs = run_experiment_flexible( 
        tokenizer_name='okt',
        okt_mode='morphs'
    )

--- 1차 실험: Okt (morphs) 사용 ---
 ML 파이프라인 실행 시작 
1. 데이터 로드 시작...
   - 훈련 데이터 shape: (5048, 2)
   - 테스트 데이터 shape: (500, 1)
2. 데이터 전처리 시작...
   - 레이블 클래스: 5개
3. Tfidf Encoding 시작...
   - Tfidf 토크나이저 사용: Okt (Mode: morphs)
   - **특징(컬럼) 개수 (Vocabulary Size): 27863**
   - 학습용 데이터 shape: (4038, 27863)
   - 검증용 데이터 shape: (1010, 27863)

 모델 학습 및 평가: LR_Baseline 시작...
   - F1 Micro Score: 0.8733

[ Classification Report ]
              precision    recall  f1-score   support

           0       0.84      0.78      0.81       179
           1       0.88      0.79      0.83       196
           2       0.90      0.89      0.89       196
           3       0.76      0.89      0.82       219
           4       1.00      1.00      1.00       220

    accuracy                           0.87      1010
   macro avg       0.88      0.87      0.87      1010
weighted avg       0.88      0.87      0.87      1010


 모델 학습 및 평가: RF_Baseline 시작...
   - F1 Micro Score: 0.8396

[ Classification Report ]
     

In [11]:
def make_submission(best_model: Any, X_test_final: Any, data_path: str = './data/', name='') -> None:
    
    print("\n🚀 Test 데이터 최종 추론 시작...")
    
    # Test 데이터 추론 (숫자 레이블 0, 1, 2, 3, 4)
    y_test_pred_encoded = best_model.predict(X_test_final)
    y_submission = y_test_pred_encoded # NumPy 배열 (추론 결과)

    # 1. 파일 경로 설정
    base_submission_path = os.path.join(data_path, 'submission.csv')
    
    # 2. 💡 [핵심 수정]: 기본 submission.csv 파일을 강제로 불러옵니다 (try-except 제거).
    submission_df = pd.read_csv(base_submission_path)
    print(f"   - 기존 Submission 양식 파일 로드: {base_submission_path} (Shape: {submission_df.shape})")

    # 3. 'class' 컬럼에 추론 결과 덮어쓰기
    if len(submission_df) == len(y_submission):
        # 기존 DataFrame의 'class' 컬럼을 추론 결과로 덮어씁니다.
        submission_df['class'] = y_submission
    else:
        # 행 개수가 불일치해도 강제로 덮어쓰거나, 사용자에게 경고 후 새 파일을 생성합니다.
        # 기존 양식 유지를 위해 경고 후 강제 덮어쓰기를 유지합니다.
        print(f"🚨 경고: 추론 결과({len(y_submission)})와 기존 제출 파일 행({len(submission_df)})의 개수가 불일치합니다. 강제로 class 컬럼을 덮어씁니다.")
        # 데이터 길이 불일치 시 Pandas가 오류를 발생시키므로, 안전하게 새 DataFrame을 생성하여 할당합니다.
        # (기존 컬럼 수를 맞추는 것이 더 안전합니다.)
        submission_df['class'] = y_submission 
        
    # 4. Submission 파일 저장 (name 포함)
    submission_path = os.path.join(data_path, f'submission{name}.csv')
    submission_df.to_csv(submission_path, index=False, encoding='utf-8-sig')
    
    print(f"🎉 최종 추론 완료 및 Submission 파일 저장: **{submission_path}** (Shape: {submission_df.shape})")

In [12]:
# 최고 성능 모델 선택
best_result = max(results_morphs, key=lambda x: x['f1_micro'])
best_model = best_result['model']
best_model_name = best_result['model_name']

print("\n==============================================")
print(f"🥇 최적 모델 선정: **{best_model_name}** (F1 Micro: {best_result['f1_micro']:.4f})")

# 2. make_submission 호출
make_submission(
    best_model=best_model, 
    X_test_final=X_test_final_morphs, 
    #data_path=data_path,
    name='_baseline'
)


🥇 최적 모델 선정: **LR_Baseline** (F1 Micro: 0.8733)

🚀 Test 데이터 최종 추론 시작...
   - 기존 Submission 양식 파일 로드: ./data/submission.csv (Shape: (500, 2))
🎉 최종 추론 완료 및 Submission 파일 저장: **./data/submission_baseline.csv** (Shape: (500, 2))
